In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-of-tweets/test_samples.txt
/kaggle/input/sentiment-analysis-of-tweets/train.txt


In [2]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

In [3]:
gc.collect()

22

In [4]:
train=pd.read_csv('/kaggle/input/sentiment-analysis-of-tweets/train.txt')
train

,tweet_id,sentiment,tweet_text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out)
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney
...,...,...,...
21460,522949024132112384,neutral,"the day after newark ill be able to say """"i met demi lovato yesterday and im meeting demi lovato tomorrow"""""
21461,522372593312350209,neutral,FEC hold farewell session for seven ministers - President Goodluck Jonathan on Wednesday led the Federal Executive... http://t.co/vmj98piNp9
21462,522515200592052224,neutral,Luca Di Montezemolo (who's last day was Monday) on why Alonso is leaving Ferrari: http://t.co/ltJoCPubSc
21463,523089087155437568,positive,Coffee is pretty much the answer to all questions today. #Friday #tgif


In [5]:
test=pd.read_csv('/kaggle/input/sentiment-analysis-of-tweets/test_samples.txt')
test

,tweet_id,tweet_text
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor, margate, ocean city area. I'm just waiting for the coordinator to hopefully call me tomorrow"
1,218775148495515649,"Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not"
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt"
4,171874368908050432,"Excuse the connectivity of this live stream, from Baba Amr, so many activists using only one Sat Modem. LIVE http://t.co/U283IhZ5 #Homs"
...,...,...
5393,210378118865756160,It's a Wednesday girls night out as '90's band Wilson Phillips fills the Winspear Opera House with their brand of top 40 joy at 8p....
5394,245177521304399872,"night college course sorted, just have to enrole tomorrow now.. finally starting to figure out what i wanna do :-) #busyday"
5395,259280987089932288,For the 1st time in 30 years. For your splendiferous entertainment. Arts in Lechlade proudly presents an OLD... http://t.co/Qg0XaxTV
5396,201113950211940352,NURSES DAY - 12 MAY 2012. Nursing: The heart beat of the health


In [6]:
print(train.shape)
train.head()

(21465, 3)


,tweet_id,sentiment,tweet_text
0,264183816548130816,positive,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)
1,263405084770172928,negative,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.
2,262163168678248449,negative,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.
3,264249301910310912,negative,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out)
4,262682041215234048,neutral,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney


In [7]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
train['sentiment']= label_encoder.fit_transform(train['sentiment']) 
  
train['sentiment'].unique()

array([2, 0, 1])

In [8]:
train.head()

,tweet_id,sentiment,tweet_text
0,264183816548130816,2,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)
1,263405084770172928,0,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.
2,262163168678248449,0,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.
3,264249301910310912,0,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out)
4,262682041215234048,1,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney


In [9]:
test=pd.read_csv('/kaggle/input/sentiment-analysis-of-tweets/test_samples.txt')
print(test.shape)
test.head()

(5398, 2)


,tweet_id,tweet_text
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor, margate, ocean city area. I'm just waiting for the coordinator to hopefully call me tomorrow"
1,218775148495515649,"Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not"
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt"
4,171874368908050432,"Excuse the connectivity of this live stream, from Baba Amr, so many activists using only one Sat Modem. LIVE http://t.co/U283IhZ5 #Homs"


In [10]:
test['sentiment']=-999
test.head()

,tweet_id,tweet_text,sentiment
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor, margate, ocean city area. I'm just waiting for the coordinator to hopefully call me tomorrow",-999
1,218775148495515649,"Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not",-999
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6,-999
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt",-999
4,171874368908050432,"Excuse the connectivity of this live stream, from Baba Amr, so many activists using only one Sat Modem. LIVE http://t.co/U283IhZ5 #Homs",-999


In [11]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

(26863, 3)


,tweet_id,sentiment,tweet_text
26858,210378118865756160,-999,It's a Wednesday girls night out as '90's band Wilson Phillips fills the Winspear Opera House with their brand of top 40 joy at 8p....
26859,245177521304399872,-999,"night college course sorted, just have to enrole tomorrow now.. finally starting to figure out what i wanna do :-) #busyday"
26860,259280987089932288,-999,For the 1st time in 30 years. For your splendiferous entertainment. Arts in Lechlade proudly presents an OLD... http://t.co/Qg0XaxTV
26861,201113950211940352,-999,NURSES DAY - 12 MAY 2012. Nursing: The heart beat of the health
26862,237999067286876160,-999,We have 15 minutes left until the 2nd episode of Styled To Rock. #UKNavi #RihannaNavy


In [12]:

gc.collect()

100

In [13]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
nltk.download('punkt')
nltk.download('wordnet')
import re

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [14]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [15]:
df['clean_text']=clean_review(df.tweet_text.values)
df.head()

,tweet_id,sentiment,tweet_text,clean_text
0,264183816548130816,2,Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :),gas by my house hit i u m going to chapel hill on sat
1,263405084770172928,0,Theo Walcott is still shit\u002c watch Rafa and Johnny deal with him on Saturday.,theo walcott is still shit u c watch rafa and johnny deal with him on saturday
2,262163168678248449,0,its not that I\u2019m a GSP fan\u002c i just hate Nick Diaz. can\u2019t wait for february.,it not that i u m a gsp fan u c i just hate nick diaz can u t wait for february
3,264249301910310912,0,Iranian general says Israel\u2019s Iron Dome can\u2019t deal with their missiles (keep talking like that and we may end up finding out),iranian general say israel u s iron dome can u t deal with their missile keep talking like that and we may end up finding out
4,262682041215234048,1,Tehran\u002c Mon Amour: Obama Tried to Establish Ties with the Mullahs http://t.co/TZZzrrKa via @PJMedia_com No Barack Obama - Vote Mitt Romney,tehran u c mon amour obama tried to establish tie with the mullah http t co tzzzrrka via pjmedia com no barack obama vote mitt romney


In [16]:
df_train=df[df.sentiment!=-999]
df_train.shape

(21465, 4)

In [17]:
df_test=df[df.sentiment==-999]
df_test.drop('sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()

(5398, 3)


,tweet_id,tweet_text,clean_text
21465,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor, margate, ocean city area. I'm just waiting for the coordinator to hopefully call me tomorrow",jjuueellzz down in the atlantic city ventnor margate ocean city area i m just waiting for the coordinator to hopefully call me tomorrow
21466,218775148495515649,"Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not",musical awareness great big beautiful tomorrow ha an ending now is the time doe not
21467,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6,on radio fm fri oct labour analyst shawn hattingh cosatu s role in the context of unrest in the mining http t co pjzzl
21468,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt",kapan sih lo ngebuktiin jan ngomong doang susah susah usaha aja blm udh nyerah inget if you never try you ll never know cowok kok gentle bgt
21469,171874368908050432,"Excuse the connectivity of this live stream, from Baba Amr, so many activists using only one Sat Modem. LIVE http://t.co/U283IhZ5 #Homs",excuse the connectivity of this live stream from baba amr so many activist using only one sat modem live http t co u ihz homs


In [18]:
del df
gc.collect()

20

In [19]:
train_text=df_train.clean_text.values
test_text=df_test.clean_text.values
target=df_train.sentiment.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

(21465,) (21465,) (21465, 3)


In [20]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(17172,) (17172, 3)
(4293,) (4293, 3)


In [21]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

33218

In [22]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN


1133

In [23]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 3
num_classes=3

In [24]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [25]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

(17172, 1133) (4293, 1133) (5398, 1133)


In [26]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3321800   
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          42240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 3)                 99        
Total params: 3,376,555
Trainable params: 3,376,555
Non-trainable params: 0
_________________________________________________________________


In [27]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
135/135 [==============================] - 1484s 11s/step - loss: 0.9543 - accuracy: 0.5061 - val_loss: 0.8343 - val_accuracy: 0.6073
Epoch 2/3
135/135 [==============================] - 1517s 11s/step - loss: 0.6802 - accuracy: 0.7056 - val_loss: 0.8035 - val_accuracy: 0.6294
Epoch 3/3
135/135 [==============================] - 1511s 11s/step - loss: 0.4705 - accuracy: 0.8174 - val_loss: 0.9040 - val_accuracy: 0.6198
CPU times: user 1h 46min 8s, sys: 17min 8s, total: 2h 3min 16s
Wall time: 1h 15min 51s


In [28]:
y_pred1=model1.predict_classes(X_test,verbose=1)

169/169 [==============================] - 185s 1s/step


In [29]:
test.head()

,tweet_id,tweet_text,sentiment
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor, margate, ocean city area. I'm just waiting for the coordinator to hopefully call me tomorrow",-999
1,218775148495515649,"Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not",-999
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6,-999
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt",-999
4,171874368908050432,"Excuse the connectivity of this live stream, from Baba Amr, so many activists using only one Sat Modem. LIVE http://t.co/U283IhZ5 #Homs",-999


In [30]:
test['sentiment']=pred_Y=label_encoder.inverse_transform(y_pred1)
sub=test.drop(['tweet_text'],axis=1)
sub.to_csv('sub1.csv',index=False)
sub.head()

,tweet_id,sentiment
0,264238274963451904,neutral
1,218775148495515649,positive
2,258965201766998017,neutral
3,262926411352903682,neutral
4,171874368908050432,negative


In [31]:
model2=Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1133, 100)         3321800   
_________________________________________________________________
dropout (Dropout)            (None, 1133, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1133, 64)          19264     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                

In [32]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
135/135 [==============================] - 9s 68ms/step - loss: 0.9994 - accuracy: 0.4805 - val_loss: 0.9120 - val_accuracy: 0.5740
Epoch 2/3
135/135 [==============================] - 9s 64ms/step - loss: 0.7900 - accuracy: 0.6523 - val_loss: 0.7939 - val_accuracy: 0.6350
Epoch 3/3
135/135 [==============================] - 9s 67ms/step - loss: 0.4805 - accuracy: 0.8175 - val_loss: 0.8585 - val_accuracy: 0.6254
CPU times: user 33.9 s, sys: 2.55 s, total: 36.5 s
Wall time: 30.9 s


In [33]:
y_pred2=model2.predict_classes(X_test, verbose=1)

169/169 [==============================] - 1s 4ms/step


In [34]:
test['sentiment']=pred_Y=label_encoder.inverse_transform(y_pred2)
sub=test.drop(['tweet_text'],axis=1)
sub.to_csv('sub2.csv',index=False)
sub.head()

,tweet_id,sentiment
0,264238274963451904,neutral
1,218775148495515649,positive
2,258965201766998017,neutral
3,262926411352903682,neutral
4,171874368908050432,negative


In [35]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(3,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1133, 100)         3321800   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1133, 64)          19264     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 566, 64)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 566, 64)           0         
_________________________________________________________________
gru (GRU)                    (None, 566, 128)          74496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 566, 128)          0         
_________________________________________________________________
flatten (Flatten)            (None, 72448)            

In [36]:
%%time
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
135/135 [==============================] - 14s 102ms/step - loss: 1.0137 - accuracy: 0.4457 - val_loss: 0.9353 - val_accuracy: 0.5141
Epoch 2/3
135/135 [==============================] - 14s 101ms/step - loss: 0.7756 - accuracy: 0.6447 - val_loss: 0.8025 - val_accuracy: 0.6357
Epoch 3/3
135/135 [==============================] - 14s 102ms/step - loss: 0.4693 - accuracy: 0.8158 - val_loss: 0.9174 - val_accuracy: 0.6264
CPU times: user 42.8 s, sys: 1.57 s, total: 44.3 s
Wall time: 43.6 s


In [37]:
y_pred3=model3.predict_classes(X_test, verbose=1)

169/169 [==============================] - 2s 12ms/step


In [38]:
test['sentiment']=pred_Y=label_encoder.inverse_transform(y_pred3)
sub=test.drop(['tweet_text'],axis=1)
sub.to_csv('sub3.csv',index=False)
sub.head()

,tweet_id,sentiment
0,264238274963451904,neutral
1,218775148495515649,positive
2,258965201766998017,neutral
3,262926411352903682,neutral
4,171874368908050432,negative


In [39]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(3, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1133, 100)         3321800   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 1133, 100)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               176640    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 771       
Total params: 3,499,211
Trainable params: 3,499,211
Non-trainable params: 0
_________________________________________________________________


In [40]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
135/135 [==============================] - 28s 208ms/step - loss: 0.9965 - accuracy: 0.4778 - val_loss: 0.9246 - val_accuracy: 0.5544
Epoch 2/3
135/135 [==============================] - 27s 201ms/step - loss: 0.7588 - accuracy: 0.6673 - val_loss: 0.8299 - val_accuracy: 0.6131
Epoch 3/3
135/135 [==============================] - 27s 202ms/step - loss: 0.4762 - accuracy: 0.8153 - val_loss: 0.9271 - val_accuracy: 0.6091
CPU times: user 1min 12s, sys: 10.5 s, total: 1min 22s
Wall time: 1min 26s


In [41]:
y_pred4=model4.predict_classes(X_test, verbose=1)

169/169 [==============================] - 6s 36ms/step


In [42]:
test['sentiment']=pred_Y=label_encoder.inverse_transform(y_pred4)
sub=test.drop(['tweet_text'],axis=1)
sub.to_csv('sub4.csv',index=False)
sub.head()

,tweet_id,sentiment
0,264238274963451904,neutral
1,218775148495515649,positive
2,258965201766998017,neutral
3,262926411352903682,negative
4,171874368908050432,negative


In [43]:
sub_all=pd.DataFrame({'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4})
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

,model1,model2,model3,model4
0,1,1,1,1
1,2,2,2,2
2,1,1,1,1
3,1,1,1,0
4,0,0,0,0


In [44]:
pred_mode=[int(i) for i in pred_mode]

In [45]:
test['sentiment']=pred_Y=label_encoder.inverse_transform(pred_mode)
sub=test.drop(['tweet_text'],axis=1)
sub.to_csv('sub_mode.csv',index=False)
sub.head()

,tweet_id,sentiment
0,264238274963451904,neutral
1,218775148495515649,positive
2,258965201766998017,neutral
3,262926411352903682,neutral
4,171874368908050432,negative
